In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [5]:
from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///products.db")

In [14]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-8b-8192"
)
agent_executor = create_sql_agent(
    llm,
    db=db,
    agent_type="zero-shot-react-description",
    verbose=True
)

In [15]:
type(agent_executor)

langchain.agents.agent.AgentExecutor

In [16]:
agent_executor.get_prompts

<bound method Runnable.get_prompts of AgentExecutor(name='SQL Agent Executor', verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': "sql_db_query - Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.\nsql_db_schema - Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3\nsql_db_list_tables - Input is an empty string, output 

In [17]:
agent_executor.invoke(
    "List the top 2 products by price"
)



> Entering new SQL Agent Executor chain...
Thought: To answer this question, I need to know the tables and columns available in the database. I'll start by listing all the tables in the database.

Action: sql_db_list_tables
Action Input: (empty string)productsLet's get started!

Action: sql_db_list_tables
Action Input: (empty string)productsAction: sql_db_schema
Action Input: products
CREATE TABLE products (
	id INTEGER, 
	name TEXT NOT NULL, 
	url TEXT, 
	description TEXT, 
	stock_items INTEGER NOT NULL, 
	price REAL NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from products table:
id	name	url	description	stock_items	price
1	I Phone 15 pro max	http://res.cloudinary.com/dujmaxztw/image/upload/v1710849551/dypzg5qwma5k2t24nr9c.jpg	This is the latest I phone 15 pro max mobile	2	190000.0
2	Redmi Note 9 pro	http://res.cloudinary.com/dujmaxztw/image/upload/v1710849551/dypzg5qwma5k2t24nr9c.jpg	This is redmi note 9 pro latest global version mobile	9	20000.0
3	calculator	http://res.cloudinary.co

{'input': 'List the top 2 products by price',
 'output': 'The top 2 products by price are Samsung S23 Ultra with a price of 210000.0 and Samsung Galaxy S22 with a price of 200000.0.\n\nNote: The query executed on the database returns the top 2 products with the highest price.'}

In [18]:
response = agent_executor.invoke("what is the price of redmi mobile")



> Entering new SQL Agent Executor chain...
I'll start by thinking...

Thought: To find the price of a Redmi mobile, I need to query a database that contains information about mobile phones. Since I have access to SQL database tools, I'll try to use sql_db_list_tables to get a list of tables in the database that might contain this information.

Action: sql_db_list_tables
Action Input: (empty string)productsAction: sql_db_schema
Action Input: products
CREATE TABLE products (
	id INTEGER, 
	name TEXT NOT NULL, 
	url TEXT, 
	description TEXT, 
	stock_items INTEGER NOT NULL, 
	price REAL NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from products table:
id	name	url	description	stock_items	price
1	I Phone 15 pro max	http://res.cloudinary.com/dujmaxztw/image/upload/v1710849551/dypzg5qwma5k2t24nr9c.jpg	This is the latest I phone 15 pro max mobile	2	190000.0
2	Redmi Note 9 pro	http://res.cloudinary.com/dujmaxztw/image/upload/v1710849551/dypzg5qwma5k2t24nr9c.jpg	This is redmi note 9 pro latest glo

In [19]:
print(response["output"])

The price of the Redmi Note 9 pro is 20000.0.


In [20]:
agent_executor.invoke("Exlain the product samsung galaxy s22")



> Entering new SQL Agent Executor chain...
Thought: The Samsung Galaxy S22 is a smartphone, so I need to gather information about it. I can start by querying the database to get the specifications and features of the phone.

Action: sql_db_query
Action Input: SELECT * FROM smartphones WHERE model = 'Samsung Galaxy S22';Error: (sqlite3.OperationalError) no such table: smartphones
[SQL: SELECT * FROM smartphones WHERE model = 'Samsung Galaxy S22';]
(Background on this error at: https://sqlalche.me/e/20/e3q8)Thought: Oh no! It looks like the table "smartphones" does not exist in the database. I need to check the list of tables to see if the table exists.

Action: sql_db_list_tables
Action Input: (empty string)productsLet's continue from here.

Thought: It looks like the table "smartphones" does not exist in the database, but I do see a table called "products" in the list of tables. I'm going to query the schema of the "products" table to see if it contains information about the Samsung 

{'input': 'Exlain the product samsung galaxy s22',
 'output': 'The Samsung Galaxy S22 is a smartphone that is part of the Samsung Galaxy series, with a price of $200,000 and a description of "This is samsung galaxy S22".'}

## Under the hood

In [22]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
context = toolkit.get_context()
tools = toolkit.get_tools()

In [26]:
context

{'table_info': '\nCREATE TABLE products (\n\tid INTEGER, \n\tname TEXT NOT NULL, \n\turl TEXT, \n\tdescription TEXT, \n\tstock_items INTEGER NOT NULL, \n\tprice REAL NOT NULL, \n\tPRIMARY KEY (id)\n)\n\n/*\n3 rows from products table:\nid\tname\turl\tdescription\tstock_items\tprice\n1\tI Phone 15 pro max\thttp://res.cloudinary.com/dujmaxztw/image/upload/v1710849551/dypzg5qwma5k2t24nr9c.jpg\tThis is the latest I phone 15 pro max mobile\t2\t190000.0\n2\tRedmi Note 9 pro\thttp://res.cloudinary.com/dujmaxztw/image/upload/v1710849551/dypzg5qwma5k2t24nr9c.jpg\tThis is redmi note 9 pro latest global version mobile\t9\t20000.0\n3\tcalculator\thttp://res.cloudinary.com/dujmaxztw/image/upload/v1710849551/dypzg5qwma5k2t24nr9c.jpg\tcasio fx caculator\t2\t1200.0\n*/',
 'table_names': 'products'}